In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.0
    Uninstalling transformers-4.21.0:
      Successfully uninstalled transformers-4.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [2]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-htjhcjte
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-htjhcjte
  Using cached transformers-4.21.0-py3-none-any.whl (4.7 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
#GPU 사용
device = torch.device("cuda:0")

In [6]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/dictpn8.xlsx')

In [9]:
chatbot_data.sample(n=10)

,comment,label
58266,이름이 방구뽕 아니 방구 뽕 이라고 부르니까 더웃긴거같음 방 구뽕...,0
17606,저렇게 시켜야 되는거야 뭔데 자꾸 시켜 어우 말좀 이쁘게 해라 아주 명령조야,1
110618,드라마 보는 내내 눈물이 났는데 이유를 모르겠더라구요 이 글을 보니까 비로소 그...,1
69664,아우 보는내내 나도 모르게 웃고 있어서 입이 아프군,0
59455,청춘시대로 처음 알게됐는데 너무 매력있어서 빠져있네요,0
70518,요즘 왜 우영우 우영우 하는지 알겠네요,2
31180,연기자들 카리스마 쩔 어,2
97023,그래도 피해의식이 있진 않죠,1
58116,어떻게 사람이름이 방구뽕,0
85392,메이킹 좀 길게 많이 올려쥬세요오,0


In [10]:
chatbot_data.loc[(chatbot_data['label'] == "0"), 'Emotion'] = 0  #중립 => 0
chatbot_data.loc[(chatbot_data['label'] == "1"), 'Emotion'] = 1  #부정 => 1
chatbot_data.loc[(chatbot_data['label'] == "2"), 'Emotion'] = 2 #긍정 => 2

data_list = []
for q, label in zip(chatbot_data['comment'], chatbot_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [11]:
print(data_list[0])
print(data_list[300])
print(data_list[600])
print(data_list[1500])
print(data_list[2000])
print(data_list[2500])
print(data_list[-1])

['  딱딱한 무몀무 기여버', '1']
['  그러게 영우야    아빠한테 김밥을 왜 사다드려    김밥을    ', '2']
['  연기가 거를타선이 없음    ', '2']
['  나만 러브라인 좋아하나   이렇게 맑은 럽라 첨인것같아          우영우 시즌  엔딩은 둘이 커플되는거엿음좋겟음        영우가 감정적으로 성장하는것도 보고시퓜', '2']
['  저희 반 애들이 다 우영우 인사 따라해욬   계속 들려', '1']
['  정확히 현실을 꼬집는 대사다 로스쿨 수석졸업에 만점 가까이 받아도 개인변호사 사무실에서 조차 면접기회를 주지않는다  실제 우영우 같은 변호사가 한국에 있다한들 개인이든 법인이든 자폐증 있는 변호사에게 사건을 맞길까  작가가 현실과 이상을 절묘하게 표현하고 있다 ', '2']
['  털보도 별로야 그냥 새로운 인물 나오길    권모술수는 인성에서 아웃이고 털보는 최수연이 너무아까움    우변 아버지는 걍 말도안되고', '1']


In [12]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [13]:
print(len(dataset_train))
print(len(dataset_test))

84939
28314


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [20]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1328 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.1243171691894531 train acc 0.375
epoch 1 batch id 201 loss 0.5779602527618408 train acc 0.5713619402985075
epoch 1 batch id 401 loss 0.4609159827232361 train acc 0.7043329177057357
epoch 1 batch id 601 loss 0.20453917980194092 train acc 0.7707206738768719
epoch 1 batch id 801 loss 0.1659349650144577 train acc 0.8092033395755306
epoch 1 batch id 1001 loss 0.23540832102298737 train acc 0.8361638361638362
epoch 1 batch id 1201 loss 0.3045367896556854 train acc 0.8554069525395503
epoch 1 train acc 0.864930603094195


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/443 [00:00<?, ?it/s]

epoch 1 test acc 0.9555614038895641


In [25]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("중립이")
            elif np.argmax(logits) == 1:
                test_eval.append("부정이")
            elif np.argmax(logits) == 2:
                test_eval.append("긍정이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '종료하기' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 이야...이래서 우영우, 우영우 하는구나...매 화가 기다려지고, 재밌는 드라마다. 뻔한 로맨스도 없고, 매화마다 반전이 있고, 인물들의 심리 묘사 등등, 모든게 완벽해


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 고래도 아닌데 불쑥 떠오른다는 대사.. 진짜 아름답다. 작가님이 어떤분이신지 참 궁금하다
>> 입력하신 내용에서 긍정이 느껴집니다.


하고싶은 말을 입력해주세요 : 이거 도박판 나올때 배경음악 갑분 타짜되는거 겁나 웃김 ㅋㅋㅋ
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 회차를 거듭할수록 배우 라인업이.장난없네요.
>> 입력하신 내용에서 중립이 느껴집니다.


